API KEYS

# Twitter API credentials (replace with your own keys)
API_KEY = "9ZuSoAvtCKX5mnlgKHPtTG6Pp"
API_SECRET = "RGyBf7k2iVuCvsQCSvg9xfaw2NEdieGcNTiAcpnnoPQAWU3DfI"
ACCESS_TOKEN = "1595122876658565120-L5ISVpHz7mJ1DfoeQMlCQmvLTBIGxu"
ACCESS_SECRET = "nf5QIwEdZQJFfpNdqAl5qMNMR0WEooDiWAGNErzWEzove"
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAPzAxAEAAAAAQEDH%2FfozCYgxohhLy0P5ZWWIeSw%3DFglmRspmzQXuRCpqGxlQ5EUJOhvwctA9pUWqIucfN25jgYxPgW"


In [16]:
!pip install tweepy


In [ ]:
import tweepy
import pandas as pd
import re

# Twitter API credentials (replace with your own keys)
API_KEY = "JgylaJVi2bVgurr5ZdlVmSf9G"
API_SECRET = "oToTBcuxMbu9h8jKw1Hx9Fowwfiuoe2EUWgZCknIClVnYpINiU"
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAD4xxQEAAAAA1OMllFonuTYDDGFb01ra2Y%2FWl%2FQ%3D31tFcGxtXZGi5S80elbwJCzciqlqW3O2VpPx6yPoHkzR9O9NXN'
# Function to clean tweets
def clean_tweet(tweet):
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)  # Remove URLs
    tweet = re.sub(r'\@\w+|\#', '', tweet)                                    # Remove mentions and hashtags
    tweet = re.sub(r'[^\w\s]', '', tweet)                                     # Remove punctuation
    return tweet.strip()

# Function to fetch tweets using the v2 API
def get_tweets_v2(keyword, count=100):
    client = tweepy.Client(bearer_token=BEARER_TOKEN)
    tweets = client.search_recent_tweets(
        query=keyword,
        max_results=min(count, 100),  # Max results per request is 100
        tweet_fields=["created_at", "text", "lang"]
    )

    if tweets.data:
        # Extract relevant data
        data = [[tweet.created_at, tweet.text] for tweet in tweets.data if tweet.lang == "en"]
        df = pd.DataFrame(data, columns=["Timestamp", "Tweet"])
        df["Cleaned_Tweet"] = df["Tweet"].apply(clean_tweet)
        return df
    else:
        print("No tweets found for the given keyword.")
        return pd.DataFrame(columns=["Timestamp", "Tweet", "Cleaned_Tweet"])

# Example: Fetching tweets related to "Tesla"
if __name__ == "__main__":
    keyword = "Tesla"
    tweets_df = get_tweets_v2(keyword, count=50)  # Adjust count as needed
    print(tweets_df.head())


                  Timestamp  \
0 2024-11-26 13:50:27+00:00   
1 2024-11-26 13:50:27+00:00   
2 2024-11-26 13:50:27+00:00   
3 2024-11-26 13:50:27+00:00   
4 2024-11-26 13:50:26+00:00   

                                               Tweet  \
0  RT @cb_doge: BREAKING: Tesla excluded from EV ...   
1  RT @libsoftiktok: This is insane. Tesla is the...   
2  RT @DefiyantlyFree: The party who is telling y...   
3  @theshortcut @Tesla_56 Worth the wait. 🍎and it...   
4  RT @SteveInmanUIC: Bless Ya with Tesla\nHighli...   

                                       Cleaned_Tweet  
0  RT  BREAKING Tesla excluded from EV Buyer Cred...  
1  RT  This is insane Tesla is the largest auto m...  
2  RT  The party who is telling you that Donald T...  
3  Worth the wait and its ecosystem willingly ass...  
4  RT  Bless Ya with Tesla\nHighlights \n\nEdison...  


In [5]:
from textblob import TextBlob

# Function to analyze sentiment
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    return analysis.sentiment.polarity

# Apply sentiment analysis
tweets_df["Sentiment"] = tweets_df["Cleaned_Tweet"].apply(analyze_sentiment)

# Label sentiment as positive, neutral, or negative
tweets_df["Sentiment_Label"] = tweets_df["Sentiment"].apply(
    lambda x: "Positive" if x > 0 else ("Negative" if x < 0 else "Neutral")
)

print(tweets_df.head())


                  Timestamp  \
0 2024-11-26 13:50:27+00:00   
1 2024-11-26 13:50:27+00:00   
2 2024-11-26 13:50:27+00:00   
3 2024-11-26 13:50:27+00:00   
4 2024-11-26 13:50:26+00:00   

                                               Tweet  \
0  RT @cb_doge: BREAKING: Tesla excluded from EV ...   
1  RT @libsoftiktok: This is insane. Tesla is the...   
2  RT @DefiyantlyFree: The party who is telling y...   
3  @theshortcut @Tesla_56 Worth the wait. 🍎and it...   
4  RT @SteveInmanUIC: Bless Ya with Tesla\nHighli...   

                                       Cleaned_Tweet  Sentiment  \
0  RT  BREAKING Tesla excluded from EV Buyer Cred...        0.0   
1  RT  This is insane Tesla is the largest auto m...       -1.0   
2  RT  The party who is telling you that Donald T...        0.0   
3  Worth the wait and its ecosystem willingly ass...        0.1   
4  RT  Bless Ya with Tesla\nHighlights \n\nEdison...        0.0   

  Sentiment_Label  
0         Neutral  
1        Negative  
2         Neu

In [7]:
tweets_df["Date"] = tweets_df["Timestamp"].dt.date

# Now you can find the common dates
common_dates = set(tweets_df["Date"]).intersection(stock_data["Date"])
print(f"Common dates count: {len(common_dates)}")

Common dates count: 0


In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import yfinance as yf



In [9]:
stock_data = yf.download("TSLA", start="2023-01-01", end="2023-12-01")
stock_data["Price_Change"] = stock_data["Close"].diff()
stock_data["Movement"] = stock_data["Price_Change"].apply(lambda x: 1 if x > 0 else 0)


[*********************100%***********************]  1 of 1 completed


In [10]:
# Reset index and ensure the "Date" column is in the proper format
stock_data = stock_data.reset_index()  # Moves the index to a column
stock_data["Date"] = pd.to_datetime(stock_data["Date"]).dt.date  # Ensure "Date" is of type datetime.date


In [11]:
tweets_data = {
    "Timestamp": ["2023-01-03", "2023-01-04", "2023-01-05"],
    "Sentiment": [0.5, -0.2, 0.1],  # Example sentiments (replace with actual sentiment scores)
}
tweets_df = pd.DataFrame(tweets_data)
tweets_df["Timestamp"] = pd.to_datetime(tweets_df["Timestamp"])  # Ensure Timestamp is datetime
tweets_df["Date"] = tweets_df["Timestamp"].dt.date

In [12]:
import yfinance as yf
import pandas as pd

# Download stock data
stock_data = yf.download("TSLA", start="2023-01-01", end="2023-12-01")
stock_data["Price_Change"] = stock_data["Close"].diff()
stock_data["Movement"] = stock_data["Price_Change"].apply(lambda x: 1 if x > 0 else 0)

# Reset index and ensure the "Date" column is in the proper format
stock_data = stock_data.reset_index()  # Moves the index to a column
stock_data["Date"] = pd.to_datetime(stock_data["Date"]).dt.date  # Ensure "Date" is of type datetime.date

# Ensure single-level columns
stock_data.columns = stock_data.columns.get_level_values(0)

# Merge dataframes
merged_data = pd.merge(tweets_df, stock_data, on="Date", how="inner")


[*********************100%***********************]  1 of 1 completed


In [13]:
# Feature engineering
merged_data["Average_Sentiment"] = merged_data.groupby("Date")["Sentiment"].transform("mean")
merged_data = merged_data.drop_duplicates(subset="Date")
features = merged_data[["Average_Sentiment"]].fillna(0)
target = merged_data["Movement"]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Model training
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [15]:
# Predictions
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

